# Защита персональных данных клиентов

## Цель исследования 
Предложить алгоритм преобразования для защиты данные клиентов страховой компании «Хоть потоп». Который не позволит восстановить персональную информацию клиентов. 

## Ход исследования
1.  Загрузка данных. На данном этапе мы ознакамливаемся с данными и сделать предобработку данных, при необходимости.
2.  Умножение матриц. На данном этапе мы алгебраически обоснуем способ кодирования данных. Ответим на вопрос: изменится ли качество линейной регрессиипризнаки при умножении на обратимую матрицу? 
3.  Алгоритм преобразования. Предполагает описание и обоснование алгоритма преобразования исходных признаков.
4.  Проверка алгоритма. Принятие решения о возможности использования описанного алгоритма преобразования признаков.

## Описание данных
Данные клиентов страховой компании «Хоть потоп».


Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
    
    
Целевой признак: количество страховых выплат клиенту за последние 5 лет.
    
    
Условия задачи:
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.


## Загрузка данных
Создадим необходимое окружение для работы.

In [1]:
!pip install yellowbrick

In [2]:
!pip install pandas-profiling

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
from yellowbrick.classifier import ClassificationReport
import pandas_profiling
import matplotlib.pyplot as plt
import seaborn as sns

Загрузим данные и проанализируем.

In [4]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

In [5]:
def data_report(initial_data):
    """
   функция выводит: первые 10 строк на экран, информацию о данных методом info, 
   создает расширенный отчет с помощью profile_report
   """
    display(initial_data.head(10))
    display(initial_data.info())
    report = initial_data.profile_report(title='MPG dataset analysis')
    display(report)

In [6]:
data_report(data)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Выводы по итогу анализа данных:
1. Мы рассмотрели исходный набор данных, который содержит: 5000 наблюдений, 5 признаков. Целевой признак 'Страховые выплаты' - является категориальным, остальные признаки представлены в разных числовых форматах:int64, float64. Можно преобразовать данные в единый формат, который позволит сократить количество памяти, необходимое для хранения данных.
2. Отсутствуют пропуски в данных.
3. Присутствуют дубликаты - 147 строк. Данные дубликаты возникли вследствие обработки данных - удалены ФИО клиентов и прочие уникальные данные(паспортные, дата рождения). Подготовленные данные могут содержать дубликаты в силу совпадения признаков, характеризующих клиентов. От дубликатов избавляться не будем.
4. Есть несостыковки по указанию заработной платы. С 1 июня 2022 года МРОТ в РФ составляет 15 279 рублей. Минимальное значение заработной платы в данных - 5300(если указана заработная плата в рублях). В таких низких значениях, возможно, потерян 0.
5. Есть большое количество клиентов в возрасте 18 лет. Данное значение выбивается из общей картины распределения данных по возрасту. Нужно уточнить, почему такое количество. Возможно, пропущенные данные возраста дозаполнены значением 18.
6. Категориальные данные распределены не равномерно по классам. Значительное количество в категории 0.
7. Вид наименований колонок не соответствует общепринятому.

Перед обучением модели линейной регрессии — модели зависимости переменной x от одной или нескольких других переменных (факторов, регрессоров, независимых переменных) с линейной функцией зависимости. Необходимо, чтобы датасет содержал числовые данные и отсутствовали пропуски. Данные требования соблюдены.

Следующие действия:

Так как не требуется предобработка данных, выбор показателей для обучения модели, выбор и обучение модели машинного обучения, то ограничимся самыми простыми шагами по предобработке данных:

1. приведем наименования колонок к нижнему регистру и заменим пробелы в названиях на нижнее подчеркивание,
2. сделаем единый формат для числовых данных - float32.


## Предобработка данных
Преобразуем наименования колонок.

In [7]:
data.columns = data.columns.str.replace(" ", "_")
data.columns = data.columns.str.lower()

In [8]:
print(data.columns)

Index(['пол', 'возраст', 'зарплата', 'члены_семьи', 'страховые_выплаты'], dtype='object')


Преобразуем тип данных.

In [9]:
data['пол'] = pd.to_numeric(data['пол'], downcast='float')
data['страховые_выплаты'] = pd.to_numeric(data['страховые_выплаты'], downcast='float')
data['возраст'] = pd.to_numeric(data['возраст'], downcast='float')
data['зарплата'] = pd.to_numeric(data['зарплата'], downcast='float')
data['члены_семьи'] = pd.to_numeric(data['члены_семьи'], downcast='float')

In [10]:
data.head()

,пол,возраст,зарплата,члены_семьи,страховые_выплаты
0,1.0,41.0,49600.0,1.0,0.0
1,0.0,46.0,38000.0,1.0,1.0
2,0.0,29.0,21000.0,0.0,0.0
3,0.0,21.0,41700.0,2.0,0.0
4,1.0,28.0,26100.0,0.0,0.0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   пол                5000 non-null   float32
 1   возраст            5000 non-null   float32
 2   зарплата           5000 non-null   float32
 3   члены_семьи        5000 non-null   float32
 4   страховые_выплаты  5000 non-null   float32
dtypes: float32(5)
memory usage: 97.8 KB


### Итоги предобработки данных
Все преобразования выполнены успешно. Приступаем к рассчетной части. В которой нам нужно обосновать ответ на вопрос: признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

## Умножение матриц
Предлагается метод преобразования данных путем домнаженя матрицы признаков на произвольно сгенерированную обратимую матрицу. Матрица (А) является обратимой, если для данной матрицы существует обратная.
$$
А^-1
$$

называется обратной матрицей по отношению к матрице А, если 

$$
А А^{-1} = Е
$$

где Е — единичная матрица n-го порядка. Обратная матрица может существовать только для квадратных матриц. Размерность обратной матрицы для решения нашей задачи будет ровна (число признаков × число признаков).

Матрица обратима тогда и только тогда, когда она невырождена, то есть её определитель не равен нулю.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)
- $y$ — вектор целевого признака
- $P$ — матрица, на которую умножаются признаки - 
- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)
- $а$ — предсказание целевого признака

- $X1$ — преобразованная матрица признаков
- $w1$ — новый вектор весов линейной регрессии (нулевой элемент равен сдвигу)
- $а1$ — новое предсказание целевого признака
- $Е$ — единичная матрица

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

**Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?**

**Ответ:** не изменится.

**Обоснование:**

Предсказания:

$$
a = Xw
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Матрица признаков, домноженная на произвольно сгенерированную обратимую матрицу, будет иметь вид:

$$
X1 = XP      
$$

Подставив X1 в формулу рассчета вектора весов линейной регрессии получим значение w1.

$$
w1 = (X1^T X1)^{-1} X1^T y = ((XP)^T XP)^{-1} (XP)^T y = (X^TP^TXP)^{-1}X^TP^Ty = ((P^T(X^TX))P)^{-1}X^TP^Ty = P^{-1}(P^T(X^TX))^{-1}X^TP^Ty = P^{-1}(X^TX)^{-1}(P^T)^{-1}X^TP^Ty = P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y
$$


Видим, что 

$$
(P^T)^{-1} P^T = Е; 
$$

$$
(X^T X)^{-1}X^T y = w 
$$

Преобразуем выражение:

$$
w1 = P^{-1} w Е = P^{-1} w
$$

Для преобразованной матрицы X1 предсказание будет иметь вид: 

$$
a1 = X1w1
$$

Подставим в данное выражение полученное значение w1 и X1:

$$
a1 = X1w1 = XP P^{-1} w
$$


Так как

$$
P P^{-1}= Е
$$

получаем:

$$
a1 = X Е w = Xw
$$

Так как предсказание для исходной матрицы признаков равно
$$
a = Xw
$$

тогда
$$
a1 = a
$$


ч.т.д.


### Вывод по итогу умножения матриц
Нам удалось теоретически доказать, что предсказания полученные на основании исходной матрицы признаков и предсказания, полученные на основании закодированной матрицы признаков (исходной матрицы, домноженной на произвольно сгенерированную обратимую матрицу) не будут отличаться.

Следующий шаг: сформулировать алгоритм преобразования исходной матрицы признаков.



## Алгоритм преобразования
**Алгоритм**

1. Сгенерируем матрицу P - обратимую матрицу, которая может существовать только для квадратных матриц и её определитель не равен нулю.
2. Вычислим определитель (детерминант) матрицы (докажем что он не равен 0, в противном случае не будет выполнен п.1).
3. Создадим закодированную матрицу признаков - X1.
4. Найдем метрики: модели,  обученнойна исходных признаках(Х),  и модели,  обученнойна на закодированных признаках(X1).
5. Сравним метрики между собой. Если они равны, то данный алгоритм применим для кодирования.

**Обоснование**

**п.1** Пусть P есть n×n - матрица над полем скаляров F.  Если E - единичная n×n - матрица, то

$$
PE=P=EP
$$

Квадратная матрица называется обратимой, если существует матрица B, удовлетворяющая условиям

$$
PB=E, BP=E.
$$

Матрица В, удовлетворяющая этим условиям называется обратной к P. Матрицы P и B называются взаимнообратными.
Предложение: Если матрица P обратима, то существует только одна матрица, обратная к P.

Доказательство: Предположим, что B и С - матрицы обратные к P. Тогда

$$
PC=E=BP и B=BE=B(PC)=(BP)C=EC =C
$$

т.е

$$
B=C
$$

Если матрица P обратима, то обратная к P матрица обозначается 

$$
P^{-1}
$$

Таким образом, для любой обратимой матрицы выполняются равенства

$$
PP^{-1}=E, P^{-1}P=E
$$

ч.т.д.


**п.2**  Квадратная матрица P обратима (имеет обратную матрицу) тогда и только тогда, когда она невырожденная, то есть detP≠0.Если матрица P обратима, то PB=E для некоторой матрицы B. Тогда, если квадратные матрицы одного и того же порядка, то 

$$
detPB=detP⋅detB:
$$

$$
1=detE=detPB=detP⋅detB
$$
следовательно,

$$
detA≠0,detB≠0.
$$

**п.3**  𝑋1=𝑋𝑃 где матрица P  является квадратной, обратима и имеет размерность (число признаков X × число признаков X), в таком случае будет получена закодированная матрица размер которой будет равен исходной матрице(Х).

**п.4** Вычислим метрики для исходной матрицы признаков и для закодированной. Определим R^2 (коэффициент детерминации) функцию оценки регрессии. Оценка R2 – очень важный показатель, который используется для оценки производительности модели машинного обучения на основе регрессии, также известен как коэффициент детерминации. Суть его работы заключается в измерении количества отклонений в прогнозах, объясненных набором данных. Проще говоря, это разница между выборками в наборе данных и прогнозами, сделанными моделью. Чем R^2 ближе к 1, тем лучше работает модель,чем R^2 ближе к 0 - тем хуже. 

**п.5** В случае, если мы получим одинаковые значения для моделей обученных как на матрице Х, так и на матрице X1, то мы на практике докажем возможность кодирования с использованием умножения матрицы признаков на обратимую матрицу.

### Вывод
Был сформулирован алгоритм преобразования данных:
1. Генерация матрицы обратимой матрицы.
2. Вычисление определителя (детерминанта) обратимой матрицы.
3. Создание закодированной матрицы признаков.
4. Нахождение метрик: модели, обученнойна исходных признаках, и модели, обученнойна на закодированных признаках.
5. Сравнение метрик между собой.


Далее будет осуществлена проверка алгоритмов.

## Проверка алгоритма
Выделим признаки и целевой признак.

In [12]:
features = data.drop('страховые_выплаты',axis=1)
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)# добавляем константный столбец к признакам
y = data['страховые_выплаты']# целевой признаки

Разделим на тренировочные и тестовые выборки.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

Обучим модель линейной регрессии на исходных данных.

In [14]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(X_train, y_train)
R2_LR_data = r2_score(y_test, pipeline.predict(X_test))
print("R2 =", R2_LR_data)

R2 = 0.4352275398556046


Создадим функцию преобразования матрицы признаков.

In [15]:
n = X.shape[1]# количество признаков
try:
    cipher_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(cipher_matrix)
    if det != 0:
        print(cipher_matrix)
except:
    print("Матрица не является обратимой")

[[3 4 2 2 6]
 [6 1 4 2 8]
 [6 9 9 6 6]
 [4 1 1 9 4]
 [6 1 4 6 4]]


In [16]:
def cipher_features(features,cipher_matrix):
    """
    функция принимает признаки и обратимую матрицу, возвращает закодированную матрицу признаков
    """
    crypted_features = features @ cipher_matrix
    return crypted_features

In [17]:
X1 = cipher_features(X,cipher_matrix)

Выведем данные до преобразования и после.

In [18]:
print('Вид исходной матрицы признаков', '\n', features, '\n')

print('Вид матрицы обратимой', '\n',cipher_matrix, '\n')

print('Вид закодированных данных', '\n', X1)

Вид исходной матрицы признаков 
       пол  возраст  зарплата  члены_семьи
0     1.0     41.0   49600.0          1.0
1     0.0     46.0   38000.0          1.0
2     0.0     29.0   21000.0          0.0
3     0.0     21.0   41700.0          2.0
4     1.0     28.0   26100.0          0.0
...   ...      ...       ...          ...
4995  0.0     28.0   35700.0          2.0
4996  0.0     34.0   52400.0          1.0
4997  0.0     20.0   33900.0          2.0
4998  1.0     22.0   32700.0          3.0
4999  1.0     28.0   40600.0          1.0

[5000 rows x 4 columns] 

Вид матрицы обратимой 
 [[3 4 2 2 6]
 [6 1 4 2 8]
 [6 9 9 6 6]
 [4 1 1 9 4]
 [6 1 4 6 4]] 

Вид закодированных данных 
 [[198661.  49975.  49979. 446656. 198664.]
 [152285.  38419.  38420. 342284. 152286.]
 [ 84177.  21265.  21263. 189176.  84180.]
 ...
 [135735.  34086.  34090. 305234. 135734.]
 [130959.  32906.  32916. 294454. 130958.]
 [162583.  40858.  40862. 365578. 162586.]]


Данные закодированы  - трудно понять, какие данные хранятся. Разобьем данные на тренировочную и обучающую выборку и проверим R^2 значение.

In [19]:
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.25, random_state=12345)

In [20]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(X1_train, y_train)
R2_LR_data_cipher = r2_score(y_test, pipeline.predict(X1_test))
print("R2 =", R2_LR_data_cipher)

R2 = 0.4352275398556341


### Итоги проверки алгоритма
В результате выполненных действий нами были получены одинаковые значения R^2( до 11 знака после запятой) как для модели обученной на исходных наборах данных, так и на закодированных. На основании этого можно сделать вывод о том, что алгоритм работает и подтверждает алгебраический вывод в отношении неизменности предсказаний в отношении закодированной матрицы признаков по отношению к исходной. Алгоритм кодирования предполагает умножение исходной матрицы признаков на произвольно сгенерированную обратимую матрицу признаков.

## Общий вывод

Нами был выполнен анализ данных:

Были рассмотренных данных, состоящих из 5000 наблюдений и 5 признаков. В данных отсутствовали пропуски. Минимальные значния в данных по уровню зароботной платы и возрасту клиентов - требуют уточнения. Категориальные целевые данные распределены не равномерно по классам. Значительное количество в категории 0. Вид наименований колонок не соответствует общепринятому. Числовые данные хранятся в разных форматах.

Была выполнена предобработка данных:
1. приведы наименования колонок к нижнему регистру и заменены пробелы в названиях на нижнее подчеркивание,
2. сделан единый формат для числовых данных - float32.

На этапе умножения матриц теоретически доказали, что предсказания, полученные на основании исходной матрицы признаков и предсказания, полученные на основании закодированной матрицы признаков (исходной матрицы, домноженной на произвольно сгенерированную обратимую матрицу) не будут отличаться.

Был сформулирован алгоритм преобразования исходной матрицы признаков:
1. Генерация матрицы обратимой матрицы.
2. Вычисление определителя (детерминанта) обратимой матрицы.
3. Создание закодированной матрицы признаков.
4. Нахождение метрик: модели, обученнойна исходных признаках, и модели, обученнойна на закодированных признаках.
5. Сравнение метрик между собой.

В результате проверки алгоритма преобразования нами были получены одинаковые значения R^2( до 13 знака после запятой) как для модели обученной на исходных наборах данных, так и на закодированных.

In [21]:
print("Значение R2 для матрицы Х1 - ",R2_LR_data_cipher,'\n',"Значение R2 для матрицы Х - ",R2_LR_data)

Значение R2 для матрицы Х1 -  0.4352275398556341 
 Значение R2 для матрицы Х -  0.4352275398556046


На основании полученных данных, был сделан вывод о том, что алгоритм работает и подтверждает алгебраический вывод в отношении неизменности предсказаний в отношении закодированной матрицы признаков по отношению к исходной. Алгоритм кодирования предполагает умножение исходной матрицы признаков на произвольно сгенерированную обратимую матрицу признаков.